In [17]:
import os
import glob
import json
import uuid
import time
from typing import List, Dict

import pandas as pd
import requests
from qdrant_client import QdrantClient, models


## 1) 連線資訊（Embed API / LLM API / Qdrant）

- Qdrant：`http://localhost:6333`
- Embed API：你提供的 `/embed`（回傳 4096 維 embeddings）
- LLM（Query Rewrite）：使用你測通的 `/v1/chat/completions` 與 model id


In [18]:
# ===== Qdrant =====
QDRANT_URL = "http://localhost:6333"
COLLECTION_NAME = "cw03_docs"
VECTOR_SIZE = 4096

# ===== Embed API =====
EMBED_URL = "https://ws-04.wade0426.me/embed"
EMBED_TASK_DESC = "檢索技術文件"   # 依你課堂設定
EMBED_NORMALIZE = True

# ===== LLM for Query Rewrite =====
LLM_BASE = "https://ws-03.wade0426.me"
LLM_API = f"{LLM_BASE}/v1/chat/completions"

# 建議你用「動態抓 model id」避免打錯
def get_first_model_id() -> str:
    r = requests.get(f"{LLM_BASE}/v1/models", timeout=20)
    r.raise_for_status()
    data = r.json()["data"]
    return data[0]["id"]

MODEL_ID = get_first_model_id()
MODEL_ID


'/models/gpt-oss-120b'

## 2) 讀取 5 個 txt 並切塊（Chunking）

這裡先用「固定字數切塊」的方式，優點是穩定、容易 debug。  
如果你想改成「用空行分段」，我們之後再換 chunk function。

In [19]:
DATA_DIR = "data"
TXT_GLOB = os.path.join(DATA_DIR, "data_*.txt")

def read_all_txt_files(pattern: str) -> List[str]:
    paths = sorted(glob.glob(pattern))
    texts = []
    for p in paths:
        with open(p, "r", encoding="utf-8") as f:
            texts.append(f.read())
    return texts, paths

def chunk_text(text: str, chunk_size: int = 500, overlap: int = 80) -> List[str]:
    text = text.strip()
    if not text:
        return []
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + chunk_size, len(text))
        chunk = text[start:end].strip()
        if chunk:
            chunks.append(chunk)
        if end == len(text):
            break
        start = max(0, end - overlap)
    return chunks

raw_texts, paths = read_all_txt_files(TXT_GLOB)
len(paths), [os.path.basename(p) for p in paths]


(5,
 ['data_01.txt', 'data_02.txt', 'data_03.txt', 'data_04.txt', 'data_05.txt'])

## 3) 產生 documents（chunk 清單）與 metadata

我們會把每個 chunk 存進 Qdrant payload：
- source_file：來源檔名
- chunk_id：同一檔案內的 chunk 編號
- text：chunk 內容（用於回傳給 LLM 或檢查）

In [20]:
documents = []
metas = []

for p, text in zip(paths, raw_texts):
    chunks = chunk_text(text, chunk_size=500, overlap=80)
    fname = os.path.basename(p)
    for i, ch in enumerate(chunks):
        documents.append(ch)
        metas.append({"source_file": fname, "chunk_id": i})

len(documents), metas[0], documents[0][:50]

(10,
 {'source_file': 'data_01.txt', 'chunk_id': 0},
 '今天是2月2日星期一，台中市迎來了一個涼爽而舒適的清晨。凌晨時分，氣溫約在攝氏16度左右，空氣中帶著')

## 4) 呼叫 Embed API 產生 doc_embeddings（4096 維）

doc_embeddings 會是一個 list，長度要等於 documents 數量。  
每個元素是一個 4096 維的 float list。

In [21]:
def get_embeddings(texts: List[str]) -> List[List[float]]:
    r = requests.post(
        EMBED_URL,
        json={
            "texts": texts,
            "task_description": EMBED_TASK_DESC,
            "normalize": EMBED_NORMALIZE
        },
        timeout=120
    )
    r.raise_for_status()
    return r.json()["embeddings"]

def batch_embed(all_texts: List[str], batch_size: int = 32) -> List[List[float]]:
    out = []
    for i in range(0, len(all_texts), batch_size):
        batch = all_texts[i:i+batch_size]
        emb = get_embeddings(batch)
        out.extend(emb)
        print(f"[{i+len(batch)}/{len(all_texts)}] embedded")
    return out

doc_embeddings = batch_embed(documents, batch_size=32)
len(doc_embeddings), len(doc_embeddings[0])


[10/10] embedded


(10, 4096)

## 5) 建立 Qdrant collection（Dense-only, 4096 dim）

CW03 不做 hybrid，所以只建立單一向量欄位。  
distance 我們用 COSINE（常見 embedding 檢索）。

In [22]:
client = QdrantClient(url=QDRANT_URL)

client.recreate_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=models.VectorParams(
        size=VECTOR_SIZE,
        distance=models.Distance.COSINE,
    ),
)

print("✅ collection recreated:", COLLECTION_NAME)


✅ collection recreated: cw03_docs


/tmp/ipykernel_3443/3950388618.py:3: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


## 6) Upsert points 到 Qdrant

這裡用 `PointStruct` 避免你剛剛遇到的：
`dict object has no attribute 'id'`  
（新版 client 期待物件，不一定吃 dict）

In [23]:
points = []
for idx, (doc, emb, meta) in enumerate(zip(documents, doc_embeddings, metas)):
    points.append(
        models.PointStruct(
            id=uuid.uuid4().hex,
            vector=emb,
            payload={
                "text": doc,
                **meta
            }
        )
    )

client.upsert(collection_name=COLLECTION_NAME, points=points)
print("✅ upsert ok:", len(points))


✅ upsert ok: 10


## 7) Query Rewrite（讀 Prompt_ReWrite.txt）

我們會把原始問題丟給 LLM，讓它輸出「一行改寫後的搜尋語句」。  
注意：我們會做 `splitlines()[0]`，確保只取第一行避免模型碎念。

In [24]:
PROMPT_PATH = "Prompt_ReWrite.txt"

with open(PROMPT_PATH, "r", encoding="utf-8") as f:
    prompt_system = f.read().strip()

def rewrite_query(original_question: str) -> str:
    resp = requests.post(
        LLM_API,
        json={
            "model": MODEL_ID,
            "messages": [
                {"role": "system", "content": prompt_system},
                {"role": "user", "content": original_question},
            ],
            "temperature": 0.2,
        },
        timeout=60
    )
    resp.raise_for_status()
    text = resp.json()["choices"][0]["message"]["content"].strip()
    return text.splitlines()[0].strip()

# quick test
rewrite_query("Google Cloud 的 N4A 虛擬機器採用哪一款處理器？")


'Google Cloud N4A 虛擬機器 使用 哪款 處理器'

## 8) Dense Retrieval：用改寫後 query 做向量檢索

流程：
1. rewrite 出新 query
2. 對新 query 做 embedding
3. 用 Qdrant search 取 top_k chunks
4. 把 chunks 組成 context（後面可接 LLM 回答，但 CW03 你目前只要到 retrieval 也行）

In [25]:
from qdrant_client import models

def dense_search(query: str, top_k: int = 5):
    # 取得 query embedding (4096-d)
    q_vec = get_embeddings([query])[0]   # <-- 你前面已經有 get_embeddings()

    # ✅ 版本相容寫法：優先 search_points，否則 query_points
    if hasattr(client, "search_points"):
        res = client.search_points(
            collection_name=COLLECTION_NAME,
            vector=q_vec,
            limit=top_k,
            with_payload=True,
        )
        # search_points 直接回傳 list[ScoredPoint]
        return res
    else:
        res = client.query_points(
            collection_name=COLLECTION_NAME,
            query=q_vec,
            limit=top_k,
            with_payload=True,
        )
        # query_points 回傳 QueryResponse，結果在 .points
        return res.points

q0 = "Google Cloud 的 N4A 虛擬機器採用哪一款處理器？"
rq0 = rewrite_query(q0)

hits = dense_search(rq0, top_k=5)

print("原始:", q0)
print("改寫:", rq0)
print("Top1 keys:", hits[0].payload.keys())
print("Top1 source:", hits[0].payload.get("source_file"), "chunk:", hits[0].payload.get("chunk_id"))
print(hits[0].payload.get("text", "")[:200])



原始: Google Cloud 的 N4A 虛擬機器採用哪一款處理器？
改寫: Google Cloud N4A 虛擬機器 使用 的 處理器 型號是什么
Top1 keys: dict_keys(['text', 'source_file', 'chunk_id'])
Top1 source: data_04.txt chunk: 2
on處理器，內建Arm Neoverse N3平臺的運算核心，整合Google發展的動態資源管理技術，以及Titanium網路與儲存工作卸載技術。

到了今年1月底，Google Cloud宣布N4A正式上線，相較於去年底預覽版僅供應4個區域：us-central1（愛荷華州）、us-east4（北維吉尼亞州）、 europe-west3（法蘭克福）、europe-west4（荷蘭），現在擴及us


## 9) 讀取 Re_Write_questions.csv 並批次跑「改寫 + 檢索」

你作業如果要求交付結果，可以把：
- 原始問題
- 改寫後問題
- top_k 取到的 chunk（或只存 top1）
輸出成新的 csv / json 方便檢查。

In [26]:
rwq_df = pd.read_csv("Re_Write_questions.csv")
rwq_df.head(3)

,conversation_id,questions_id,questions,answer,source
0,1,1,Google Cloud 的 N4A 虛擬機器有什麼特色？,NaN,NaN
1,1,2,它跟上一代的 C4A 有什麼不同？,NaN,NaN
2,1,3,那目前可以在哪些地區使用這個服務？,NaN,NaN


## 10) (Step 4) 用 Retrieval 結果 + LLM 生成答案

流程：
1. 對每個問題做 rewrite
2. 用 rewrite 後的 query 去 Qdrant 取 top_k chunks
3. 把 chunks 組成 context 丟給 LLM
4. 回傳 answer，並把使用到的來源（source_file / chunk_id）記錄下來

In [27]:
def answer_with_context(question: str, hits, max_ctx_chars: int = 4000) -> dict:
    """
    hits: qdrant 回傳的點列表（top_k）
    回傳：{"answer": str, "sources": list[dict]}
    """

    # 1) 把 top_k chunks 組成 context（含來源）
    ctx_lines = []
    sources = []
    total = 0

    for h in hits:
        payload = h.payload or {}
        text = (payload.get("text") or "").strip()
        src = payload.get("source_file")
        cid = payload.get("chunk_id")

        # 記錄來源
        sources.append({"source_file": src, "chunk_id": cid})

        line = f"[{src} | chunk {cid}]\n{text}\n"
        if total + len(line) > max_ctx_chars:
            break
        ctx_lines.append(line)
        total += len(line)

    context = "\n---\n".join(ctx_lines).strip()

    # 2) 給 LLM 的 system prompt（可依老師風格微調）
    prompt_system_answer = (
        "你是一個助教型問答系統。"
        "只能根據我提供的 context 回答，不要編造。"
        "若 context 不足以回答，請回答：『資料不足，無法從文件中確認。』"
        "回答用繁體中文，簡潔但完整。"
    )

    # 3) 送出 LLM
    user_content = f"""問題：
{question}

可用文件片段（context）：
{context}
"""

    resp = requests.post(
        LLM_API,
        json={
            "model": MODEL_ID,
            "messages": [
                {"role": "system", "content": prompt_system_answer},
                {"role": "user", "content": user_content},
            ],
            "temperature": 0.2,
        },
        timeout=120
    )
    resp.raise_for_status()
    answer = resp.json()["choices"][0]["message"]["content"].strip()

    return {"answer": answer, "sources": sources}


## 11) (Step 4) 單題測試：Rewrite → Retrieval → LLM Answer


In [28]:
q0 = "Google Cloud 的 N4A 虛擬機器採用哪一款處理器？"

rq0 = rewrite_query(q0)                 # 你原本就有
hits = dense_search(rq0, top_k=5)       # 你原本就有

result = answer_with_context(q0, hits)

print("原始問題:", q0)
print("改寫:", rq0)
print("\n=== Answer ===")
print(result["answer"])
print("\n=== Sources ===")
print(result["sources"])


原始問題: Google Cloud 的 N4A 虛擬機器採用哪一款處理器？
改寫: Google Cloud N4A 虛擬機器 使用 哪款 處理器

=== Answer ===
Google Cloud 的 N4A 虛擬機器採用的是 **Google 自行研發的 Axion 處理器**（最新一代的 Google Axion，內建 Arm Neoverse N3 平臺的運算核心）。

=== Sources ===
[{'source_file': 'data_04.txt', 'chunk_id': 2}, {'source_file': 'data_04.txt', 'chunk_id': 0}, {'source_file': 'data_04.txt', 'chunk_id': 1}, {'source_file': 'data_05.txt', 'chunk_id': 2}, {'source_file': 'data_05.txt', 'chunk_id': 1}]


## 12) (Step 5) 逐對話批次跑：Rewrite → Retrieval → LLM Answer → 回填 CSV（含對話接續）

重點：
- `conversation_id` 相同代表同一段對話，要把前面問答當作「對話脈絡」帶入下一題。
- 逐列產生：
  - `answer`：LLM 的最終回答
  - `source`：本次回答用到的來源（source_file / chunk_id），存成 JSON 字串方便交作業檢查
- 產出檔案：`questions_answer.csv`

In [30]:
import json
import pandas as pd

# 讀取作業檔
rwq_df = pd.read_csv("Re_Write_questions.csv")

# 保險：如果 answer/source 欄不存在就補上
if "answer" not in rwq_df.columns:
    rwq_df["answer"] = ""
if "source" not in rwq_df.columns:
    rwq_df["source"] = ""

def format_history_text(history_pairs, max_turns=6) -> str:
    """
    history_pairs: [(q, a), (q, a), ...]
    只取最近 max_turns 回合，避免脈絡太長
    """
    use_pairs = history_pairs[-max_turns:]
    lines = []
    for i, (q, a) in enumerate(use_pairs, 1):
        lines.append(f"Q{i}: {q}")
        lines.append(f"A{i}: {a}")
    return "\n".join(lines)

def rewrite_with_dialogue(original_q: str, history_pairs) -> str:
    """
    把對話脈絡一起給 rewrite（同 conversation_id 接續的核心）
    你原本 rewrite_query() 只吃單題，這裡用包裝方式做到「接續」。
    """
    hist = format_history_text(history_pairs, max_turns=6).strip()
    if hist:
        # 把脈絡 + 現在問題 組成一個輸入，交給 rewrite_query() 生成更合理的改寫
        combined = f"對話脈絡：\n{hist}\n\n現在使用者的問題：\n{original_q}"
        return rewrite_query(combined)
    else:
        return rewrite_query(original_q)

def run_one_row(q: str, history_pairs):
    # 1) rewrite（含對話脈絡）
    rq = rewrite_with_dialogue(q, history_pairs)

    # 2) retrieval
    hits = dense_search(rq, top_k=5)

    # 3) answer（用原始問題 q + hits）
    result = answer_with_context(q, hits)

    # sources 存成 JSON 字串
    sources_json = json.dumps(result["sources"], ensure_ascii=False)

    return rq, result["answer"], sources_json

# ========== 逐對話批次跑 ==========
total = len(rwq_df)
ok, fail = 0, 0

# 依 conversation_id 分組（同一組就是同一段對話）
for conv_id, group_idx in rwq_df.groupby("conversation_id").groups.items():
    history_pairs = []  # 這裡累積同一段對話的 (question, answer)

    # group_idx 是原 df 的 index 列表，要按 questions_id 排序才像“對話順序”
    sub = rwq_df.loc[list(group_idx)].sort_values("questions_id")

    for idx, row in sub.iterrows():
        q = str(row["questions"])

        try:
            rq, ans, sources_json = run_one_row(q, history_pairs)

            rwq_df.at[idx, "answer"] = ans
            rwq_df.at[idx, "source"] = sources_json

            # 更新對話脈絡：下一題會用到上一題的 Q/A
            history_pairs.append((q, ans))

            ok += 1
            print(f"[{ok}/{total}] OK  conv={conv_id} qid={row['questions_id']}")

        except Exception as e:
            fail += 1
            print(f"[FAIL] conv={conv_id} qid={row.get('questions_id')} err={type(e).__name__}: {e}")

# 輸出交作業用檔案
out_path = "questions_answer.csv"
rwq_df.to_csv(out_path, index=False, encoding="utf-8-sig")

print("\n✅ 產出完成：", out_path)
print("成功筆數:", ok)
print("失敗筆數:", fail)

rwq_df.sort_values(["conversation_id", "questions_id"]).head(10)


/tmp/ipykernel_3443/2562415613.py:70: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Google Cloud 的 N4A 系列虛擬機器具備以下特色：

1. **硬體與架構**  
   - 採用 Google 自研的 **Axion 處理器**，內建 **Arm Neoverse N3** 平台運算核心。  
   - 整合 Google 的 **動態資源管理**、**Titanium 網路與儲存卸載** 技術。

2. **高性價比與能源效率**  
   - 相較於傳統 x86 虛擬機，提供 **約兩倍的 price‑performance**。  
   - 每瓦效能領先 **最高可達 80 %**，能源使用效率顯著提升。  
   - 在不同工作負載的性價比提升幅度：  
     - 計算密集型工作負載 **+5 %** 效能。  
     - MySQL 資料庫每秒交易量 **+20 %**。  
     - Java 應用（SPECjbb2015）成本效益 **+85 %**。  
     - 網站伺服器（Nginx 反向代理）領先 **+90 %**。  
     - CPU 整數運算（SPECrate2017）性價比 **達 205 %**。

3. **多樣化工作負載支援**  
   - 微服務、容器化應用、開源資料庫、批次處理、資料分析、軟體開發環境、實驗測試、資料準備、網站服務等皆適用，亦可用於各種 AI 應用。

4. **地域與平台支援**  
   - 已在多個區域上線（美國、歐洲、亞洲等），並支援 GKE（Autopilot 與標準模式），使用 GKE 1.34.1‑gke.3403001 以上版本即可選擇 N4A。

綜合以上，N4A 以 Arm 架構提供更佳的效能、成本與能源表現，且適用範圍廣泛，是 Google Cloud 目前最具成本效益的 N 系列一般用途虛擬機器。' has dtype incompatible with float64, please explic

[1/6] OK  conv=1 qid=1
[2/6] OK  conv=1 qid=2
[3/6] OK  conv=1 qid=3
[4/6] OK  conv=2 qid=1
[5/6] OK  conv=2 qid=2
[6/6] OK  conv=2 qid=3

✅ 產出完成： questions_answer.csv
成功筆數: 6
失敗筆數: 0


,conversation_id,questions_id,questions,answer,source
0,1,1,Google Cloud 的 N4A 虛擬機器有什麼特色？,Google Cloud 的 N4A 系列虛擬機器具備以下特色：\n\n1. **硬體與架構...,"[{""source_file"": ""data_04.txt"", ""chunk_id"": 2}..."
1,1,2,它跟上一代的 C4A 有什麼不同？,N4A 與前一代的 C4A 在定位與形態上有以下幾個主要差異：\n\n| 項目 | C4A（...,"[{""source_file"": ""data_04.txt"", ""chunk_id"": 2}..."
2,1,3,那目前可以在哪些地區使用這個服務？,目前 N4A 服務已在以下 Google Cloud 區域可供使用：\n\n- **us‑c...,"[{""source_file"": ""data_04.txt"", ""chunk_id"": 2}..."
3,2,1,最近去日本旅遊天氣要注意什麼？,根據目前的資訊，近期前往日本旅遊需要留意以下天氣與健康狀況：\n\n1. **寒冷與強風**...,"[{""source_file"": ""data_03.txt"", ""chunk_id"": 0}..."
4,2,2,那邊現在有流感疫情嗎？,根據提供的資料，日本目前正處於第二波流感流行。國立健康危機管理研究機構的報告顯示，截至1月2...,"[{""source_file"": ""data_03.txt"", ""chunk_id"": 0}..."
5,2,3,東京的情況嚴重嗎？,根據林氏璧的報告，東京的流感情況正呈上升趨勢。全日本 47 個都道府縣中，東京所在的關東地區...,"[{""source_file"": ""data_03.txt"", ""chunk_id"": 0}..."
